<a href="https://colab.research.google.com/github/Conv-AI/TTS-Dev/blob/main/TTS_finetuning_Fluttershy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up the environment

In [7]:
!pip install wandb
!wandb login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [1]:
BRANCH = 'main'
# If you're using Google Colab and not running locally, uncomment and run this cell.
!apt-get install sox libsndfile1 ffmpeg
!pip install wget unidecode pynini==2.1.4
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-5zgcd5mv/nemo-toolkit_dc6a4651a4034c06945623f556364994
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-5zgcd5mv/nemo-toolkit_dc6a4651a4034c06945623f556364994
  Using cached matplotlib-3.5.2-cp37-cp3

In [2]:
!pip install hydra-core==1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
home_path = !(echo $HOME)
home_path = home_path[0]
print(home_path)

/root


In [5]:
#=== load the repo and data (Thanks Synthbot) ===
!sudo apt -qq install -y sox
!git clone "https://github.com/synthbot-anon/synthbot.git" /content/synthbot
!(cd /content/synthbot; git checkout experimental)
!pip3 install pysoundfile 
!pip3 install librosa

sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
fatal: destination path '/content/synthbot' already exists and is not an empty directory.
Already on 'experimental'
Your branch is up to date with 'origin/experimental'.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Preparing the Dataset for Fastpitch finetuning

In [6]:
#downloading the dataset
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=13OpYwSBjmbVbaqmbEcOe1YmdYPH049ox' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=13OpYwSBjmbVbaqmbEcOe1YmdYPH049ox" -O Fluttershy.tar && rm -rf /tmp/cookies.txt

--2022-06-17 13:10:17--  https://docs.google.com/uc?export=download&confirm=t&id=13OpYwSBjmbVbaqmbEcOe1YmdYPH049ox
Resolving docs.google.com (docs.google.com)... 173.194.218.102, 173.194.218.139, 173.194.218.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.218.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/clf8sol28dbprqsmgserc1lqgcbtcc6f/1655471400000/05434657448262868821/*/13OpYwSBjmbVbaqmbEcOe1YmdYPH049ox?e=download [following]
--2022-06-17 13:10:17--  https://doc-14-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/clf8sol28dbprqsmgserc1lqgcbtcc6f/1655471400000/05434657448262868821/*/13OpYwSBjmbVbaqmbEcOe1YmdYPH049ox?e=download
Resolving doc-14-84-docs.googleusercontent.com (doc-14-84-docs.googleusercontent.com)... 142.251.107.132, 2607:f8b0:400c:c32::84
Connecting to doc-14-84-docs.googleusercontent.com (doc-

In [7]:
skip_noisy = True # Disable to train with Noisy Data included
allowed_emotions = """
Anxious
Angry
Annoyed
Amused
Confused
Crazy
Disgust
Exhausted
Fear
Happy
Neutral
Sad
Serious
Singing
Shouting
Surprised
Smug
Love
Sarcastic
Tired
Whispering
Whining
""".split("\n")[1:-1]

In [8]:
percentage_training_data = 0.95 # 90% of Data will be used for training, 5% for Validation

In [9]:
import sys
sys.path.append('/content/synthbot/src')
from ponysynth.corpus import ClipperArchive, phoneme_transcription
import librosa
import subprocess

In [10]:
archive_fn = "/content/Fluttershy.tar"
archive = ClipperArchive(archive_fn)

In [11]:
data_path = 'Fluttershy'
allowed_emotions = [x.lower() for x in allowed_emotions]
!mkdir {data_path}
!mkdir {data_path+"/out"}
all_clips = []; all_clips_arpa = []; skipped_count=0; too_short_count=0; emotion_skip=0
for key in archive.keys(): # write the audio files for processing in bash terminal
  audio = archive.read_audio(key)
  audio_fn = '{}/{}.wav'.format(data_path, key)
  audio_fn_ = '{}/{}.wav'.format(data_path+"/out", key)
  with open(audio_fn, 'wb') as audio_out:
    audio_out.write(audio.read())

mkdir: cannot create directory ‘Fluttershy’: File exists
mkdir: cannot create directory ‘Fluttershy/out’: File exists


In [12]:
%%script bash
# trim all 48Khz files
cd Fluttershy
for input in *.wav; do
  output="out/$input"
  sox "$input" "$output" silence 1 0.05 0.1% reverse silence 1 0.05 0.1% reverse;
done

In [13]:
import os

should_continue = 0 # should run "continue" command inside outer loop.
for key in archive.keys():
  label = archive.read_label(key)
  audio_fn = '{}/{}.wav'.format(data_path, key)
  audio_fn_ = '{}/{}.wav'.format(data_path+"/out", key)
  if (label['noise'] in ['Very Noisy','Noisy']) and skip_noisy: os.remove(audio_fn_); skipped_count+=1; continue

  for tag in label['tags']:
    if tag.lower() not in allowed_emotions:
        try: os.remove(audio_fn_)
        except: pass
        print(tag+" emotion not in list"); emotion_skip+=1; should_continue = 1; break # this is supposed to break the outer loop
  if should_continue: should_continue = 0; continue

  audio = archive.read_audio(key)
  transcript = label['utterance']['content']
  
  if os.stat(audio_fn_).st_size < 71602:
    #print("Skipping Audio, Duration: "+str(len(librosa.core.load(audio_fn_, sr=48000)[0])/48000))
    try: os.remove(audio_fn_)
    except: pass
    too_short_count+=1; continue # Skips files based on size.

  filelist_line = "{}|{}".format(audio_fn_, transcript)
  all_clips.append(filelist_line)
  filelist_line = "{}|{}".format(audio_fn_, phoneme_transcription(label))
  all_clips_arpa.append(filelist_line)
    
print(str(skipped_count)+" Files are too Noisy.")
print(str(emotion_skip)+" Files contain an emotion not in permitted emotions")
print(str(too_short_count)+" Files are too short")
print(str(len(list(archive.keys()))-(skipped_count+too_short_count+emotion_skip))+" Files kept in dataset.")

2331 Files are too Noisy.
0 Files contain an emotion not in permitted emotions
147 Files are too short
1410 Files kept in dataset.


In [14]:
# shuffle the training data
import random
random.seed(0)
random.shuffle(all_clips)

# get train, test, validation splits
num_clips = len(all_clips)
train_end = int(num_clips * percentage_training_data)

train = all_clips[:train_end]
validation = all_clips[train_end:]

train_arpa = all_clips_arpa[:train_end]
validation_arpa = all_clips_arpa[train_end:]

# dump the info to filelist files
with open('train_filelist_fluttershy.txt', 'w') as train_out:
  train_out.write('\n'.join(train)+'\n'+'\n'.join(train_arpa)+"")
with open('val_filelist_fluttershy.txt', 'w') as val_out:
  val_out.write('\n'.join(validation)+'\n'+'\n'.join(validation_arpa)+"")

In [15]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
      - Validating: OK
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json


In [3]:
#imports
import os
import json
import random
import librosa

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pathlib import Path

from nemo.collections.tts.models import FastPitchModel

[NeMo W 2022-06-17 14:55:07 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-06-17 14:55:08 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [4]:
#creating the json files for training and validation
with open('/content/train_filelist_fluttershy.txt') as f :
  records = f.readlines()

print("Number of records : ",len(records))

train_manifest = 'fastpitch_train.json'

train_rec = []
random.shuffle(records)
count = 0

for i in records :
  if count > 1200 :
    break
  
  i = i.split('|')
  audio_filepath = i[0]
  text = i[-1].strip('\n')
  #if '{' in text :
    #print(text)
    #continue
  count = count + 1

  duration = librosa.get_duration(filename=audio_filepath)
  r = {
       "audio_filepath" : audio_filepath,
       "text" : text,
       "duration" : round(duration,1),
       "text_no_preprocessing" : text
    }

  train_rec.append(r)

with open(train_manifest, "w") as f:
    for s in train_rec:
        f.write(json.dumps(s) + '\n')
        
print("Training Data : ", len(train_rec))

Number of records :  2678
Training Data :  1201


In [5]:
with open('/content/val_filelist_fluttershy.txt') as f :
  records_val = f.readlines()

print("Number of records : ", len(records_val))

val_manifest = 'fastpitch_val.json'
count = 0
val_rec = []
random.shuffle(records_val)

for i in records_val:
  if count > 50 :
    break
    
  i = i.split('|')
  audio_filepath = i[0]
  text = i[-1].strip('\n')

  # if '{' in text :
  #   print(text)
  #   continue
  count = count + 1
  duration = librosa.get_duration(filename=audio_filepath)
  r = {
       "audio_filepath" : audio_filepath,
       "text" : text,
       "duration" : round(duration,1),
       "text_no_preprocessing" : text
    }

  val_rec.append(r)

with open(val_manifest, "w") as f:
    for s in val_rec:
        f.write(json.dumps(s) + '\n')

Number of records :  142


In [19]:
FastPitchModel.from_pretrained("tts_en_fastpitch")
nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/tts_en_fastpitch_align.nemo")]
print(f"Copying {nemo_files[0]} to ./")
Path("./tts_en_fastpitch_align.nemo").write_bytes(nemo_files[0].read_bytes())

[NeMo I 2022-06-17 13:10:51 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.10.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo.
[NeMo I 2022-06-17 13:10:51 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.10.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo
[NeMo I 2022-06-17 13:10:51 common:789] Instantiating model from pre-trained checkpoint
[NeMo I 2022-06-17 13:10:53 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-06-17 13:11:12 experimental:28] Module <class 'nemo.collections.tts.torch.g2ps.IPAG2P'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-06-17 13:11:12 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-06-17 13:11:13 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_matri

[NeMo I 2022-06-17 13:11:13 features:200] PADDING: 1
[NeMo I 2022-06-17 13:11:17 save_restore_connector:243] Model FastPitchModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.10.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo.
Copying /root/.cache/torch/NeMo/NeMo_1.10.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo to ./


187023360

In [20]:
!wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/fastpitch_finetune.py

!mkdir -p conf \
&& cd conf \
&& wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/conf/fastpitch_align_v1.05.yaml \
&& cd ..

--2022-06-17 13:11:18--  https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/fastpitch_finetune.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1751 (1.7K) [text/plain]
Saving to: ‘fastpitch_finetune.py.1’

fastpitch_finetune. 100%[===================>]   1.71K  --.-KB/s    in 0s      

2022-06-17 13:11:18 (30.1 MB/s) - ‘fastpitch_finetune.py.1’ saved [1751/1751]

--2022-06-17 13:11:18--  https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/conf/fastpitch_align_v1.05.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 

In [21]:
# additional files
!mkdir -p tts_dataset_files && cd tts_dataset_files \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/cmudict-0.7b_nv22.01 \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/heteronyms-030921 \
&& wget wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/nemo_text_processing/text_normalization/en/data/whitelist/lj_speech.tsv \
&& cd ..

--2022-06-17 13:11:18--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/cmudict-0.7b_nv22.01
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3721964 (3.5M) [text/plain]
Saving to: ‘cmudict-0.7b_nv22.01.1’

cmudict-0.7b_nv22.0 100%[===================>]   3.55M  --.-KB/s    in 0.07s   

2022-06-17 13:11:19 (52.3 MB/s) - ‘cmudict-0.7b_nv22.01.1’ saved [3721964/3721964]

--2022-06-17 13:11:19--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/heteronyms-030921
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awa

## finetuning the Fastpitch model

In [22]:
%cd /content/
%pwd

/content


'/content'

In [1]:
# TODO(oktai15): remove +model.text_tokenizer.add_blank_at=true when we update FastPitch checkpoint
!(python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=fastpitch_train.json \
  validation_datasets=fastpitch_val.json \
  sup_data_path=fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.01 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=Fluttershy_training_logs \
  +init_from_nemo_model=tts_en_fastpitch_align.nemo \
  +trainer.max_steps=3000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=10 \
  model.train_ds.dataloader_params.batch_size=16 model.validation_ds.dataloader_params.batch_size=16 \
  model.n_speakers=1 model.pitch_mean=121.9 model.pitch_std=23.1 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

[NeMo W 2022-06-17 15:41:48 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-06-17 15:41:49 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-06-17 15:41:49 exp_manager:287] Experiments will be logged at Fluttershy_training_logs/FastPitch/2022-06-17_15-41-49
[NeMo I 2022-06-17 15:41:49 exp_manager:661] TensorboardLogger has been set up
[NeMo W 2022-06-17 15:41:49 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2319: LightningDeprecationWarning: `Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.
      rank_zero_deprecatio

## Preparing the data for HiFiGAN finetuning

## finetuning the HiFiGAN model

## Copy the final weights to the gdrive for future reference

## Load the checkpoints and do the inference